In [ ]:
# autoreload
%load_ext autoreload
%autoreload 2

from vorflow import ConceptualMesh, MeshGenerator, VoronoiTessellator
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString, box

In [ ]:

domain = box(0, 0, 200, 200)
poly1 = box(50, 50, 75, 75)
# generate sine wave curve for river
river_coords = [(x, 75 + 20 * np.sin(0.1 * x)) for x in range(-10, 220, 10)]
river_coords = LineString(river_coords)
well_point = Point(25, 25)
fault_line = LineString([(100, 0), (100, 150)])

In [ ]:
# 1. Setup Blueprint
background_lc=100

blueprint = ConceptualMesh(crs="EPSG:3857")
blueprint.add_polygon(domain, zone_id=1)#,border_density=100,dist_max_in=background_lc)
blueprint.add_polygon(poly1, zone_id=2, 
                      resolution=1, z_order=2,
                      dist_max_out=3* background_lc,
                      #dist_max_in=10
                      ) 
blueprint.add_point(well_point, point_id="Well-A", 
                    resolution=2,
                    dist_max=300)
blueprint.add_line(fault_line, line_id="Fault-1", 
                   resolution=1, 
                   is_barrier=True,
                   #straddle_width=1,
                   dist_max=3* 5)
blueprint.add_line(river_coords, line_id="riv-1", 
                   resolution=1, 
                  is_barrier=False,
                  dist_max=3* 5,)

clean_polys, clean_lines, clean_pts = blueprint.generate()

# 2. Mesh Generation
mesher = MeshGenerator(background_lc=background_lc, verbosity=10)
mesher.generate(clean_polys, clean_lines, clean_pts)

# 3. Voronoi Conversion
tessellator = VoronoiTessellator(mesher, blueprint, clip_to_boundary=True)
grid_gdf = tessellator.generate()

# 4. Output
#grid_gdf.to_file("mf6_grid.shp")



fig,ax = plt.subplots(1,1,figsize=(12,12))
ax.set_aspect('equal')
# plot domain
ax.plot(*domain.exterior.xy,marker='.', color='black')
# plot poly1
#ax.plot(*poly1.exterior.xy, color='blue',lw=1,ls='--',label='Polygon 1')
ax.plot(*river_coords.xy, color='cyan',lw=1,ls='--',label='River')
ax.plot(*fault_line.xy, color='r',lw=1,ls='--',label='Fault')
ax.scatter(*well_point.xy, color='r',s=20,marker='x',label='Well')
grid_gdf.plot(ax=ax, alpha=0.5, edgecolor='k', linewidth=0.2)

ax.legend(loc='upper left', fontsize=8)
#ax.set_ylim(50,80)
#ax.set_xlim(90, 110)
#
#ax.set_ylim(40,80)
#ax.set_xlim(40, 80)

fig.tight_layout()

In [ ]:
from vorflow.utils import *

In [ ]:
gdf = calculate_mesh_quality(grid_gdf,calc_ortho=True)
gdf.plot(column='drift_ratio', cmap='viridis', legend=True, figsize=(10,8),vmax=.25)

In [ ]:
summarize_quality(gdf)